In [14]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [15]:
batch_size = 64 # declaring the batch size

In [16]:
epochs = 100 # number of epochs to train

In [17]:
latent_dim = 256

In [18]:
num_samples = 125000

In [19]:
import os, sys

In [20]:
data_path = os.path.join(os.path.dirname(os.getcwd()), "datasets", "fra-eng", "fra.txt") # path to the corpus file
# note that it is super important to know the format of the corpus file
# in this case the format is such
# <eng-text><tab><french-translation of the same eng text>
# each new training example is in a new line

In [21]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [22]:
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split("\n") # spliting training examples

# remember we are trying to get only `sample` number of lines and not any more for training 
for line in lines[:min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split("\t")
    
    target_text = '\t' + target_text + "\n"
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    
    # including chars in input and target characters
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [23]:
# here input_characters is the total number of input characters
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)


max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples: " + str(len(input_texts))) # total number of samples that we are training
print("Number of unique input tokens: " + str(num_encoder_tokens))
print("Number of unique output tokens: " + str(num_decoder_tokens))
print("Max sequence length for inputs: " + str(max_encoder_seq_length))
print("Max sequence length for outputs: " + str(max_decoder_seq_length))

Number of samples: 1000
Number of unique input tokens: 62
Number of unique output tokens: 77
Max sequence length for inputs: 11
Max sequence length for outputs: 33


In [27]:
# note that here we are only taking the first 1000 training examples

In [30]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
# recall that input_characters is a list of all the unique input characters that we have in the 1000 training examples
# that we have read so far

# what enumerate does is that it assigns a counter to the unique characters, so basically if we get the following list
# a = ["ola", "mundo", "keras"], we would get back from enumerate 1, "ola"; 2, "mundo"; 3, "keras"

target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [32]:
# here is the important part as we are defining what goes into the input data
# max_encoder seq length is the maximum number of characters in one training example in the entire corpus
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

# the number of input_texts and the number of target_texts will be the same
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

# here we just created the vectors that we are going to use as inputs for the encoder and inputs and targets for the 
# decoder

In [34]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        # i is the first index as it is the ith training example 
        # t is the second index as the t-th character in ith training example
        encoder_input_data[i, t, input_token_index[char]] = 1
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1
        
        # Here is an important thing to understand
        ###############---------------------------IMPORTANT------------------------#####################
        # The decoder_target_data will be ahead of the decoder_input_data by one time step and this is an important factor
        # The decoder_target_data will obviously not contain the first character
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1
        

In [35]:
encoder_inputs = Input(shape=(None, num_encoder_tokens)) # num_encoder_tokens is essentially the vocab size

In [36]:
encoder = LSTM(latent_dim, return_state=True)
# Each unit or cell of LSTM within the layer has an internal cell state referred to as c and a hidden state referred
# to as h. 
# If we need to access the internal state c of the last time step we need to use return_state = True

In [37]:
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# here the encoder_outputs would be hidden state at last time step, state_h would be the hidden state of the last time step
# again and state_c is the internal state of the last time step

In [39]:
encoder_states = [state_h, state_c] # we will only keeep the states and discard the output

In [40]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

In [41]:
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
# return sequences will return the hidden states for each and every time step

In [42]:
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# note here we are passing the encoder_states as the initial_state of the decoder lstm
# decoder_outputs will actially contain the hidden states of all the time steps as an array

In [43]:
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
# Dense is an ordinary dense neural network. 

In [44]:
decoder_outputs = decoder_dense(decoder_outputs)

In [45]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [47]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [48]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# passing all the data here

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 8s 10ms/step - loss: 1.5808 - val_loss: 1.6461
Epoch 2/100
800/800 [==============================] - 5s 6ms/step - loss: 1.3729 - val_loss: 1.6564
Epoch 3/100
800/800 [==============================] - 5s 6ms/step - loss: 1.3275 - val_loss: 1.5837
Epoch 4/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2983 - val_loss: 1.5391
Epoch 5/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2623 - val_loss: 1.5412
Epoch 6/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2438 - val_loss: 1.4976
Epoch 7/100
800/800 [==============================] - 5s 6ms/step - loss: 1.2123 - val_loss: 1.4817
Epoch 8/100
800/800 [==============================] - 5s 6ms/step - loss: 1.1912 - val_loss: 1.4560
Epoch 9/100
800/800 [==============================] - 5s 6ms/step - loss: 1.1552 - val_loss: 1.4174
Epoch 10/100
800/800 [======================

Epoch 81/100
800/800 [==============================] - 5s 6ms/step - loss: 0.2083 - val_loss: 1.1697
Epoch 82/100
800/800 [==============================] - 5s 6ms/step - loss: 0.2005 - val_loss: 1.1694
Epoch 83/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1973 - val_loss: 1.1907
Epoch 84/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1993 - val_loss: 1.1878
Epoch 85/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1850 - val_loss: 1.2274
Epoch 86/100
800/800 [==============================] - 4s 6ms/step - loss: 0.1856 - val_loss: 1.2220
Epoch 87/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1816 - val_loss: 1.2353
Epoch 88/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1768 - val_loss: 1.2095
Epoch 89/100
800/800 [==============================] - 5s 6ms/step - loss: 0.1759 - val_loss: 1.2417
Epoch 90/100
800/800 [==============================] - 5s 7ms/step - loss: 0.1667

In [50]:
model.save('seq2seqKeras.h5')

C:\Users\Rajdeep\AppData\Local\conda\conda\envs\tfdeeplearning\lib\site-packages\keras\engine\topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [51]:
# Inference Model

In [52]:
encoder_model = Model(encoder_inputs, encoder_states)

In [54]:
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_state_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_state_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_state_inputs, [decoder_outputs] + decoder_states)

In [55]:
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [56]:
def decode_sequence(input_seq):
    """
    This function is for decoding a sequence of english letters
    """
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq) # this is actually encoder_input data

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    # target_sequence is simply a one hot vector of size vocab size

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # predicting one character at a time
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [60]:
for seq_index in range(112095, 112097):
    # this trick below is just for converting input_seq to a list
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq=input_seq)
    print('-')
    print("Input sentence: ", input_texts[seq_index])
    print("Decoded sentence: ", decoded_sentence)
    

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 3 array(s), but instead got the following list of 1 arrays: [array([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., ...